# Modèle de modération CIVIC-DRC
## Détection de propos injurieux, tribaux ou menaçants (spaCy + lexique multilingue)
Objectif : signaler à l'admin les textes hors normes avant publication.
Langues : français, lingala, kikongo, swahili, tshiluba.

In [1]:
# Cellule 1 — Imports et chargement du modèle spaCy (français)
# En ligne de commande : pip install spacy pandas scikit-learn joblib && python -m spacy download fr_core_news_sm

import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import re

# Charger le modèle spaCy français (tokenisation, lemmatisation)
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "fr_core_news_sm"], check=True)
    nlp = spacy.load("fr_core_news_sm")

print("spaCy chargé (fr_core_news_sm).")

spaCy chargé (fr_core_news_sm).


In [22]:
# Cellule 2 — Lexique de 50 mots/phrases à détecter (injurieux, tribal, menaçant)
# Répartis en français, lingala, kikongo, swahili, tshiluba. À compléter selon votre contexte.

LEXIQUE_A_SIGNALER = [
    # Français (injurieux / menaces)
    "fou", "idiot", "débile", "salaud", "menteur", "traître", "mort", "tuer", "violence", "haine",
    # Lingala
    "mbwa", "likoso", "kozala", "kolya", "koboma", "mpamba", "ndoki", "lisumu", "bobe", "ebele",
    # Kikongo
    "mpasi", "nfumu", "lufu", "vonda", "ndoki", "kosa", "mbwa", "nuni", "yala", "kondwa",
    # Swahili
    "kufa", "kuua", "mwizi", "mbwa", "shari", "maovu", "adui", "hasira", "dhuluma", "fitina",
    # Tshiluba
    "kufwa", "kuvonda", "buji", "bulemu", "kabidi", "mulopwe", "bwana", "mukalanga", "tabulwa", "kabanga",
]

# On s'assure d'avoir 50 entrées (certains mots peuvent être communs à plusieurs langues)
LEXIQUE_A_SIGNALER = list(dict.fromkeys(LEXIQUE_A_SIGNALER))[:50]
print(f"Lexique : {len(LEXIQUE_A_SIGNALER)} termes à détecter.")
print(LEXIQUE_A_SIGNALER)

Lexique : 47 termes à détecter.
['fou', 'idiot', 'débile', 'salaud', 'menteur', 'traître', 'mort', 'tuer', 'violence', 'haine', 'mbwa', 'likoso', 'kozala', 'kolya', 'koboma', 'mpamba', 'ndoki', 'lisumu', 'bobe', 'ebele', 'mpasi', 'nfumu', 'lufu', 'vonda', 'kosa', 'nuni', 'yala', 'kondwa', 'kufa', 'kuua', 'mwizi', 'shari', 'maovu', 'adui', 'hasira', 'dhuluma', 'fitina', 'kufwa', 'kuvonda', 'buji', 'bulemu', 'kabidi', 'mulopwe', 'bwana', 'mukalanga', 'tabulwa', 'kabanga']


In [23]:
# Cellule 3 — Construction des données d'entraînement (≈70 par langue à signaler, beaucoup d'acceptables)
# À signaler = contenant mots du lexique ou propos injurieux/tribal/menaçant. Acceptable = neutre, civique.

# --- Français (à signaler, ~70) ---
phrases_positives_fr = [
    "le president est fou", "c est un idiot", "il est débile", "salaud", "menteur", "traître",
    "il faut tuer", "mort aux autres", "violence partout", "haine entre nous", "violence gratuite",
    "espèce d idiot", "tu es fou", "ils sont débiles", "salaud de ministre", "traître à la nation",
    "on va tuer", "mort à tous", "plein de haine", "propager la haine", "appel à la violence",
    "idiot utile", "gouvernement de salauds", "menteurs pathologiques", "tous des traîtres",
    "fou dangereux", "débile mental", "violence envers", "haine tribale", "haine ethnique",
    "il est complètement fou", "quelle bande d idiots", "c est débile", "salaud va", "espèce de menteur",
    "traîtres à la patrie", "qu on les tue", "mort aux opposants", "violence politique", "haine raciale",
    "fou furieux", "idiot du village", "débile profond", "salaud de politicien", "menteur invétéré",
    "traître vendu", "tuer les gens", "mort aux étrangers", "acte de violence", "discours de haine",
    "on dirait un fou", "tu es un idiot", "c est débile comme idée", "salaud de chef", "menteur comme lui",
    "traître à son peuple", "faire tuer", "souhaiter la mort", "recourir à la violence", "semer la haine",
    "devenir fou", "traiter d idiot", "débile congénital", "vieux salaud", "gros menteur",
    "traître à la révolution", "ordre de tuer", "désir de mort", "explosion de violence", "culture de haine",
]
# Compléter à ~70
phrases_positives_fr += [
    "fou à lier", "idiot fini", "débile mental", "salaud de juge", "menteur né",
    "traître payé", "tuer sans raison", "souhaiter mort", "violence inutile", "haine gratuite",
] * 2

# --- Lingala (~70) ---
phrases_positives_ln = [
    "mbwa na yo", "kolya bato", "koboma", "ndoki", "likoso", "bobe", "lisumu", "ebele",
    "koboma bato", "mbwa", "kolya", "mpamba", "ndoki na bino", "likoso mingi", "bobe ya",
    "kozala na likoso", "koboma moto", "mbwa mabe", "ndoki ya", "lisumu na ngai",
    "bato koboma", "mbwa na bino", "kolya bato na", "koboma bato banso", "ndoki na yo",
    "likoso ezali", "bobe oyo", "lisumu ezali", "ebele na", "mpamba te",
] * 2
phrases_positives_ln += ["koboma", "mbwa", "ndoki", "likoso", "bobe", "lisumu", "kolya bato", "kufa"] * 5

# --- Kikongo (~70) ---
phrases_positives_kg = [
    "vonda", "mpasi", "lufu", "ndoki", "kosa", "mbwa", "nfumu", "yala", "kondwa",
    "vonda bantu", "mpasi mingi", "lufu na", "ndoki ya", "kosa na", "mbwa mbi",
    "bantu vonda", "mpasi na nge", "lufu ya", "ndoki na bantu", "kosa ya",
] * 3
phrases_positives_kg += ["vonda", "mpasi", "lufu", "ndoki", "kosa", "mbwa"] * 5

# --- Swahili (~70) ---
phrases_positives_sw = [
    "kufa", "kuua", "mwizi", "adui", "hasira", "maovu", "shari", "dhuluma", "fitina",
    "kuua watu", "mwizi wa", "adui ya", "hasira nyingi", "maovu ya", "kufa na",
    "watu kuua", "mwizi mkuu", "adui wa serikali", "hasira kubwa", "maovu na",
] * 3
phrases_positives_sw += ["kufa", "kuua", "mwizi", "adui", "hasira", "maovu", "fitina"] * 5

# --- Tshiluba (~70) ---
phrases_positives_ts = [
    "kufwa", "kuvonda", "buji", "bulemu", "kabidi", "mulopwe", "bwana", "tabulwa", "kabanga",
    "kuvonda bantu", "buji wa", "bulemu na", "kabidi ya", "bantu kufwa", "mulopwe na",
] * 4
phrases_positives_ts += ["kufwa", "kuvonda", "buji", "bulemu", "tabulwa"] * 6

# Regrouper tous les "à signaler" (dédupliquer un peu mais garder volume)
phrases_positives = (
    phrases_positives_fr[:70]
    + phrases_positives_ln[:70]
    + phrases_positives_kg[:70]
    + phrases_positives_sw[:70]
    + phrases_positives_ts[:70]
)
phrases_positives = list(dict.fromkeys(phrases_positives))  # garder ordre, enlever doublons
if len(phrases_positives) < 300:
    phrases_positives += (phrases_positives_fr + phrases_positives_ln)[: 350 - len(phrases_positives)]

# --- Acceptables (beaucoup plus que les à signaler pour que les neutres ne soient pas marqués) ---
phrases_negatives = [
    "le president a annoncé un projet", "améliorer les routes", "santé et éducation",
    "construction d une école", "eau potable dans le village", "formation des jeunes",
    "transparence et bonne gouvernance", "élections libres", "développement économique",
    "sécurité alimentaire", "accès aux soins", "réparation des ponts", "emploi pour tous",
    "protection de l environnement", "culture et patrimoine", "sport pour la jeunesse",
    "il faut construire des écoles", "améliorer la santé en RDC", "transparence des élections",
    "construire des hôpitaux", "routes en bon état", "électricité pour les villages",
    "éducation gratuite", "lutte contre la corruption", "justice pour tous",
    "paix et sécurité", "réconciliation nationale", "dialogue politique",
    "investissement dans l agriculture", "soutien aux jeunes", "droits des femmes",
    "assainissement des villes", "accès à l eau", "soins de santé primaire",
    "scolarisation des enfants", "formation professionnelle", "création d emplois",
    "infrastructures routières", "énergie renouvelable", "protection des forêts",
    "gouvernement ouvert", "participation citoyenne", "décentralisation",
    "budget transparent", "contrats publics visibles", "lutte anti corruption",
    "élections transparentes", "liberté de la presse", "indépendance de la justice",
    "développement durable", "économie locale", "tourisme responsable",
    "sécurité alimentaire pour tous", "nutrition des enfants", "vaccination",
    "construction de ponts", "entretien des routes", "transport public",
    "accès à internet", "numérique pour l école", "administration en ligne",
    "état civil fiable", "cadastre", "titres de propriété",
    "assainissement urbain", "gestion des déchets", "eau courante",
    "éclairage public", "espaces verts", "sécurité dans les quartiers",
    "réforme de l enseignement", "santé maternelle", "routes nationales",
    "agriculture durable", "pêche et élevage", "commerce équitable",
    "coopération internationale", "aide au développement", "projets communautaires",
    "réseau de santé", "centres de formation", "bibliothèques publiques",
    "stabilité politique", "réforme institutionnelle", "lutte contre la pauvreté",
    "égalité des chances", "accès au crédit", "microfinance",
    "innovation technologique", "emploi des jeunes", "insertion professionnelle",
    "sécurité civile", "protection sociale", "retraite et assurance",
    "urbanisation maîtrisée", "logement décent", "eau et assainissement",
    "énergies propres", "reboisement", "biodiversité",
    "éducation des filles", "alphabétisation", "formation continue",
    "recherche scientifique", "universités", "enseignement technique",
    "marchés publics transparents", "commande publique", "appels d offres",
    "justice sociale", "droits de l homme", "libertés fondamentales",
    "presse libre", "pluralisme politique", "société civile",
    "débat public", "consultation citoyenne", "pétition et proposition",
    "gestion des ressources", "fiscalité équitable", "douanes et taxes",
    "infrastructure numérique", "couverture réseau", "télécommunications",
]
# Équilibrer : au moins autant d'acceptables que d'à signaler (améliore l'accuracy)
phrases_negatives = list(dict.fromkeys(phrases_negatives))
n_pos = len(phrases_positives)
if len(phrases_negatives) < n_pos:
    phrases_negatives = (phrases_negatives * ((n_pos // len(phrases_negatives)) + 1))[: n_pos + 50]

texts = phrases_positives + phrases_negatives
labels = [1] * len(phrases_positives) + [0] * len(phrases_negatives)
df = pd.DataFrame({"texte": texts, "etiquette": labels})
print(df.head(10))
print(f"\nTotal : {len(df)} exemples ({df['etiquette'].sum()} à signaler, {len(df) - df['etiquette'].sum()} acceptables).")

                  texte  etiquette
0  le president est fou          1
1        c est un idiot          1
2         il est débile          1
3                salaud          1
4               menteur          1
5               traître          1
6          il faut tuer          1
7       mort aux autres          1
8      violence partout          1
9      haine entre nous          1

Total : 714 exemples (342 à signaler, 372 acceptables).


In [24]:
# Cellule 4 — Prétraitement avec spaCy (tokenisation, lemmatisation, nettoyage)
# On normalise le texte et on extrait les lemmes pour améliorer la détection.

def preprocess_spacy(texte, nlp_model):
    if pd.isna(texte) or not isinstance(texte, str):
        return ""
    texte = re.sub(r"\s+", " ", texte.strip().lower())
    doc = nlp_model(texte)
    lemmas = [t.lemma_ for t in doc if not t.is_stop and t.is_alpha]
    return " ".join(lemmas) if lemmas else texte

df["texte_norm"] = df["texte"].apply(lambda x: preprocess_spacy(x, nlp))
# Feature binaire : 1 si le texte contient un mot du lexique (renforce la détection)
def contient_lexique(texte_norm, lexique):
    if not texte_norm:
        return 0
    mots = set(texte_norm.split())
    return 1 if any(lex in mots for lex in lexique) else 0
df["contient_lexique"] = df["texte_norm"].apply(lambda t: contient_lexique(t, LEXIQUE_A_SIGNALER))
print("Exemples après prétraitement :")
print(df[["texte", "texte_norm", "etiquette", "contient_lexique"]].head(8))

Exemples après prétraitement :
                  texte     texte_norm  etiquette  contient_lexique
0  le president est fou  president fou          1                 1
1        c est un idiot        c idiot          1                 1
2         il est débile         débile          1                 1
3                salaud         salaud          1                 1
4               menteur        menteur          1                 1
5               traître        traître          1                 1
6          il faut tuer   falloir tuer          1                 1
7       mort aux autres           mort          1                 1


In [25]:
# Cellule 5 — Vectorisation TF-IDF des textes
# Transformation des textes en vecteurs numériques pour le classifieur.

X = df["texte_norm"]
y = df["etiquette"]

vectorizer = TfidfVectorizer(
    max_features=500,
    ngram_range=(1, 2),
    min_df=1,
    strip_accents="unicode",
    lowercase=True,
)

from scipy.sparse import hstack, csr_matrix

X_tfidf = vectorizer.fit_transform(X)
X_lex = csr_matrix(df["contient_lexique"].values.reshape(-1, 1))
X_vec = hstack([X_tfidf, X_lex])
print(f"Matrice : {X_vec.shape[0]} textes, {X_vec.shape[1]} traits (TF-IDF + 1 feature lexique).")

Matrice : 714 textes, 501 traits (TF-IDF + 1 feature lexique).


In [27]:
# Cellule 6 — Entraînement du classifieur (régression logistique)
# Prédit 1 = à signaler à l'admin, 0 = acceptable.

X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, test_size=0.25, random_state=42, stratify=y
)

clf = LogisticRegression(max_iter=500, random_state=42, class_weight='balanced')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Entraînement terminé.")
print(f"Accuracy (test) : {accuracy_score(y_test, y_pred):.2%}")

Entraînement terminé.
Accuracy (test) : 92.74%


In [28]:
# Cellule 7 — Évaluation : rapport de classification et matrice de confusion

print("Rapport de classification :")
print(classification_report(y_test, y_pred, target_names=["acceptable", "a_signer"], zero_division=0))
print("Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

Rapport de classification :
              precision    recall  f1-score   support

  acceptable       0.88      1.00      0.93        93
    a_signer       1.00      0.85      0.92        86

    accuracy                           0.93       179
   macro avg       0.94      0.92      0.93       179
weighted avg       0.94      0.93      0.93       179

Matrice de confusion :
[[93  0]
 [13 73]]


C:\Users\DEBUZE DAVID\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\DEBUZE DAVID\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\DEBUZE DAVID\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

In [29]:
# Cellule 8 — Sauvegarde du modèle, du vectoriseur et du lexique pour le backend
# Le backend Node pourra appeler un script Python qui charge ces fichiers.

import os
os.makedirs("model_moderation", exist_ok=True)

joblib.dump(clf, "model_moderation/classifier.joblib")
joblib.dump(vectorizer, "model_moderation/vectorizer.joblib")
joblib.dump(LEXIQUE_A_SIGNALER, "model_moderation/lexique.joblib")

print("Fichiers sauvegardés dans model_moderation/ : classifier.joblib, vectorizer.joblib, lexique.joblib.")

Fichiers sauvegardés dans model_moderation/ : classifier.joblib, vectorizer.joblib, lexique.joblib.


In [30]:
# Cellule 9 — Fonction de prédiction (réutilisable)
# Retourne True si le texte doit être signalé à l'admin, False sinon.
# Utilise la même feature "contient_lexique" qu'à l'entraînement.

from scipy.sparse import hstack, csr_matrix

def predire_texte(texte, nlp_model, vec, clf_model, lexique):
    """Retourne (a_signer: bool, proba: float). lexique = liste des mots à signaler."""
    if not texte or not str(texte).strip():
        return False, 0.0
    norm = preprocess_spacy(str(texte), nlp_model)
    X_tfidf = vec.transform([norm])
    mots = set(norm.split()) if norm else set()
    contient_lexique = 1 if any(lex in mots for lex in lexique) else 0
    X = hstack([X_tfidf, csr_matrix([[contient_lexique]])])
    proba = clf_model.predict_proba(X)[0][1]  # proba classe 1 (à signaler)
    a_signer = clf_model.predict(X)[0] == 1
    return bool(a_signer), float(proba)

# Recharger pour tester (en production on charge une seule fois)
clf_loaded = joblib.load("model_moderation/classifier.joblib")
vec_loaded = joblib.load("model_moderation/vectorizer.joblib")
lexique_loaded = joblib.load("model_moderation/lexique.joblib")
print("Modèle, vectoriseur et lexique rechargés.")

Modèle, vectoriseur et lexique rechargés.


In [35]:
# Cellule 10 — Tests sur des exemples (dont "le president est fou")
# Vérification que les propos injurieux ou hors normes sont bien détectés.

exemples = [
    "le president est fou",
    "il faut construire des écoles",
    "salaud et traître",
    "améliorer la santé en RDC",
    "koboma bato",
    "transparence des élections",
     "Vous travailler bien",
     "oza zoba",
    "gouvernement de menteur",
    "le ministre est fou",
     "le gouverneur est fou",
]

print("Résultats des tests :")
print("-" * 50)
for phrase in exemples:
    a_signer, proba = predire_texte(phrase, nlp, vec_loaded, clf_loaded, lexique_loaded)
    statut = "SIGNALER À L'ADMIN" if a_signer else "acceptable"
    print(f"  '{phrase}'")
    print(f"    → {statut} (proba à signaler: {proba:.2%})")
    print()

Résultats des tests :
--------------------------------------------------
  'le president est fou'
    → SIGNALER À L'ADMIN (proba à signaler: 97.25%)

  'il faut construire des écoles'
    → acceptable (proba à signaler: 7.33%)

  'salaud et traître'
    → SIGNALER À L'ADMIN (proba à signaler: 97.79%)

  'améliorer la santé en RDC'
    → acceptable (proba à signaler: 6.76%)

  'koboma bato'
    → SIGNALER À L'ADMIN (proba à signaler: 97.87%)

  'transparence des élections'
    → acceptable (proba à signaler: 6.74%)

  'Vous travailler bien'
    → acceptable (proba à signaler: 9.21%)

  'oza zoba'
    → acceptable (proba à signaler: 9.21%)

  'gouvernement de menteur'
    → SIGNALER À L'ADMIN (proba à signaler: 97.19%)

  'le ministre est fou'
    → SIGNALER À L'ADMIN (proba à signaler: 97.50%)

  'le gouverneur est fou'
    → SIGNALER À L'ADMIN (proba à signaler: 97.62%)

